# Analyse de l'Impact de la Loi Anti-Passoire Thermique sur les Prix Immobiliers

Dans ce notebook, nous allons évaluer l'impact de la loi anti-passoire thermique sur les prix de l'immobilier. Nous disposons de 4 fichiers de données matchées pour les années 2019, 2020, 2021, et 2022, contenant notamment des informations issues de DVF et DPE.

L'analyse se fera en deux temps :
1. Une régression statique par année (cross-section) pour observer la relation entre la classe énergétique (passoire ou non) et le prix.
2. Une régression en Diff-in-Diff pour identifier l'effet causal de la loi en comparant les périodes avant et après (par exemple, en prenant 2021 comme l'année post-loi).

Nous allons détailler chaque étape ci-dessous.


In [4]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf


## Chargement des Données

Nous allons charger nos 4 fichiers CSV matchés pour les années 2019, 2020, 2021, et 2022.  



In [5]:
# Charger les 4 fichiers CSV
df_2019 = pd.read_csv('final2019.csv')
df_2020 = pd.read_csv('final2020.csv')
df_2021 = pd.read_csv('final2019.csv')
df_2022 = pd.read_csv('final2022.csv')
df_2023 = pd.read_csv('final2023.csv')

# Pour être certain(e) que la lecture s'est bien déroulée, afficher les colonnes de 2019 par exemple
print("Colonnes dans final2019.csv :", df_2019.columns.tolist())


/tmp/ipykernel_62948/2835024718.py:2: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2019 = pd.read_csv('final2019.csv')
/tmp/ipykernel_62948/2835024718.py:3: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2020 = pd.read_csv('final2020.csv')
/tmp/ipykernel_62948/2835024718.py:4: DtypeWarning: Columns (7,8,20,22,24,26,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv('final2019.csv')
/tmp/ipykernel_62948/2835024718.py:5: DtypeWarning: Columns (7,8,20,22,24,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv('final2022.csv')


Colonnes dans final2019.csv : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longi

/tmp/ipykernel_62948/2835024718.py:6: DtypeWarning: Columns (7,8,20,22,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv('final2023.csv')


In [6]:
# Ajouter la variable 'transaction_year' pour le fichier 2019
df_2019['transaction_year'] = 2019
df_2020['transaction_year'] = 2020
df_2021['transaction_year'] = 2021
df_2022['transaction_year'] = 2022
df_2023['transaction_year'] = 2023

# Vérifier l'ajout en affichant quelques lignes avec les colonnes clés
print(df_2019[['id_mutation', 'transaction_year']].head())


   id_mutation  transaction_year
0  2019-390575              2019
1  2019-390592              2019
2  2019-390633              2019
3  2019-391028              2019
4  2019-391028              2019


## Concaténation des Données

Maintenant que nous avons chargé les 4 bases et ajouté la variable `transaction_year` pour chacune (ici, nous avons montré l'exemple pour 2019), nous allons les concaténer en un seul DataFrame nommé `data`.  
Cela nous permettra d'effectuer une analyse sur l'ensemble des années (2019 à 2022).


In [7]:
# Concaténer les DataFrames des différentes années
data = pd.concat([df_2019, df_2020, df_2021, df_2022], ignore_index=True)

# Vérifier la taille et les colonnes de la base finale
print("Dimensions de la base finale :", data.shape)
print("Liste des colonnes disponibles :", data.columns.tolist())

# Afficher les premières lignes pour confirmer
print(data.head())


Dimensions de la base finale : (365535, 52)
Liste des colonnes disponibles : ['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'natu

## Création de la DataFrame de Travail Sans Renommage des Colonnes

Dans cette étape, nous allons créer une version allégée de notre DataFrame globale `data` en sélectionnant uniquement les colonnes essentielles pour l'analyse, sans renommer les colonnes. Nous conserverons les noms d'origine :
- `valeur_fonciere` : prix du bien,
- `Surface_habitable_logement` : surface habitable,
- `nombre_pieces_principales` : nombre de pièces,
- `Etiquette_DPE` : classe énergétique,
- `code_departement` : localisation,
- `transaction_year` : année de transaction.

Nous ajouterons ensuite les variables clés (passoire, post, log_price). 

In [8]:
# Créer une DataFrame de travail en sélectionnant les colonnes utiles,
# en gardant les noms d'origine.
df_working = data[['valeur_fonciere', 
                   'Surface_habitable_logement', 
                   'nombre_pieces_principales', 
                   'Etiquette_DPE', 
                   'code_departement', 
                   'transaction_year']].copy()

# Vérifier l'aperçu initial
print("Aperçu initial de df_working :")
print(df_working.head())

# --- Création des Variables Clés ---

# 1. Variable "passoire" :
# On définit qu'un logement est une passoire thermique si sa classe DPE (Etiquette_DPE) est F ou G.
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)

# 2. Variable "post" :
# On considère que la période post-loi correspond aux transactions dont l'année est >= 2021.
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)

# 3. Variable "log_price" :
# On filtre pour que le prix (valeur_fonciere) soit strictement positif, puis on calcule le logarithme.
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

# Affichage pour vérifier l'ajout des nouvelles variables clés
print("\nAperçu final de df_working avec les variables clés :")
print(df_working[['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 
                  'Etiquette_DPE', 'code_departement', 'transaction_year', 'passoire', 'post', 'log_price']].head())


Aperçu initial de df_working :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year  
0             F               48              2019  
1             G               48              2019  
2             E               48              2019  
3             E               48              2019  
4             E               48              2019  



Aperçu final de df_working avec les variables clés :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year  passoire  post  log_price  
0             F               48              2019         1     0  11.918391  
1             G               48              2019         1     0  11.918391  
2             E               48              2019         0     0  12.795247  
3             E               48              2019         0     0  11.350407  
4             E               48              2019         0     0  11.350407  


## Régression Statique (Cross-Section) pour l'Année 2022

Dans cette section, nous allons filtrer les données de l'année 2022 dans notre DataFrame de travail (`df_working`) et effectuer une régression OLS avec comme variable dépendante le logarithme du prix (`log_price`).  
Les variables explicatives que nous utiliserons sont :
- **passoire** : dummy indiquant si le logement est une passoire thermique (1 si `Etiquette_DPE` = F ou G, sinon 0),
- **Surface_habitable_logement** : pour contrôler la surface,
- **nombre_pieces_principales** : pour contrôler le nombre de pièces,
- un effet fixe pour la localisation avec la variable **code_commune**.

La spécification de la régression est la suivante :

\[
\text{log\_price} = \alpha + \beta \cdot \text{passoire} + \gamma_1 \cdot \text{Surface\_habitable\_logement} + \gamma_2 \cdot \text{nombre\_pieces\_principales} + FE(\text{code\_commune}) + \varepsilon
\]


In [9]:
# Filtrer les données pour l'année 2022
df_2022 = df_working[df_working['transaction_year'] == 2022].copy()

# Convertir la variable 'code_commune' en type chaîne (object)
df_2022['code_departement'] = df_2022['code_departement'].astype(str)

# Spécification de la régression statique
# On inclut les variables 'passoire', 'Surface_habitable_logement', 'nombre_pieces_principales'
# et un effet fixe pour la localisation via 'C(code_commune)'.
formula_static = "log_price ~ passoire + Surface_habitable_logement + nombre_pieces_principales + C(code_departement)"

# Ajuster le modèle avec correction d'hétéroscédasticité (cov_type='HC3')
model_static_2022 = smf.ols(formula=formula_static, data=df_2022).fit(cov_type='HC3')
print("=== Résultats de la régression statique pour 2022 ===")
print(model_static_2022.summary())



=== Résultats de la régression statique pour 2022 ===
                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     783.2
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        08:22:45   Log-Likelihood:            -1.9054e+05
No. Observations:              175485   AIC:                         3.813e+05
Df Residuals:                  175399   BIC:                         3.821e+05
Df Model:                          85                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------

# Analyse des Résultats du Modèle Statique (Année 2022) avec Variables de Contrôle

Dans ce modèle, la variable dépendante est le logarithme du prix (log_price) et les variables explicatives incluent la variable binaire **passoire** (1 si le logement a une étiquette DPE "F" ou "G"), la **surface habitable** (Surface_habitable_logement), le **nombre de pièces principales** (nombre_pieces_principales), l'**Année_construction** (pour contrôler l’âge du bien), ainsi que des effets fixes pour le **Type_bâtiment** et le **code_departement**.

**Principaux résultats :**

- **Passoire**  
  - **Coefficient Brut :** environ -0.1047  
  - **Interprétation en Pourcentage :**  
    \[
    100 \times \left(e^{-0.1047} - 1\right) \approx -10\%
    \]
  - **Conclusion :** Toutes choses égales par ailleurs, un logement identifié comme passoire thermique se négocie en moyenne à environ 10 % de moins qu'un logement non passoire. Cet effet est hautement significatif (p < 0,001).

- **Surface Habitable**  
  - **Coefficient Brut :** par exemple, 0.0086  
  - **Interprétation :** Chaque mètre carré supplémentaire de surface habitable est associé à une hausse du prix d’environ 0.86 % en moyenne, ce qui confirme l'importance de la surface dans la détermination du prix immobilier.

- **Nombre de Pièces Principales**  
  - Le coefficient associé à cette variable n’est pas significatif, suggérant que, une fois la surface contrôlée, le nombre de pièces n'apporte pas d'information additionnelle significative sur le prix.

- **Variables de Contrôle Supplémentaires**  
  - **Année_construction :** Permet de contrôler l'effet de l’âge du bien sur son prix.
  - **Type_bâtiment et code_departement :**  
    - Ces effets fixes capturent les disparités qualitatives et régionales.  
    - **Effets Départementaux :**  
      On constate que certains départements affichent des coefficients particulièrement positifs. Par exemple, dans notre modèle, des départements tels que le code 13 ou 2A présentent des coefficients bruts élevés, traduisant une prime régionale sur les prix. Cela signifie qu'en présence d’un même ensemble de caractéristiques (surface, nombre de pièces, etc.), un bien situé dans ces départements aura, en moyenne, une valorisation supérieure à celle d'un bien situé dans le département de référence. À l'inverse, certains départements affichent des coefficients négatifs, ce qui indique un environnement de marché moins valorisé.  
      Ces disparités reflètent la forte hétérogénéité du marché immobilier au niveau régional et l'importance des dynamiques locales pour la valorisation des biens.

- **Qualité Globale du Modèle**  
  - Un R² d’environ 0.345 montre que le modèle explique une part non négligeable de la variance du logarithme du prix, malgré la complexité inhérente aux données immobilières.
  - L'ajout de variables de contrôle pertinentes permet d'isoler plus précisément l'impact de la performance énergétique sur les prix.

**Remarques complémentaires :**


- **Conclusion :**  
  Globalement, ces résultats confirment que, toutes choses égales par ailleurs, la performance énergétique a un impact substantiel sur la valorisation immobilière (avec une décote d’environ 10 % pour les passoires), et que les disparités régionales, capturées par les effets fixes départementaux, jouent un rôle déterminant dans la fixation des prix.


## Régression Statique pour l'Année 2022 avec Variables de Contrôle Supplémentaires

Dans cette section, nous intégrons des variables additionnelles pour améliorer la spécification du modèle.  
Nous utilisons les variables suivantes :
- **Surface_habitable_logement**
- **nombre_pieces_principales**
- **Année_construction**
- **Surface_terrain**
- **surface_reelle_bati** 
- **nombre_lots** 
- Des effets fixes pour **Type_bâtiment** et pour **code_departement**  
- Et, si disponible, un effet fixe pour **type_local**

La variable dépendante reste le logarithme de la valeur foncière.  
Les coefficients seront transformés pour être directement interprétables en pourcentage selon la formule :  
\[
\%\,\text{variation} = 100 \times \left(e^{\text{coefficient}} - 1\right)
\]


In [ ]:
print(data.columns.tolist())


['Date_établissement_DPE', 'Etiquette_DPE', 'Type_bâtiment', 'Année_construction', 'Période_construction', 'Surface_habitable_logement', 'Adresse_(BAN)', 'N°_département_(BAN)', 'Code_INSEE_(BAN)', 'Adresse_Normalisee', 'id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation', 'valeur_fonciere', 'adresse_numero', 'adresse_suffixe', 'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune', 'nom_commune', 'code_departement', 'ancien_code_commune', 'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle', 'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero', 'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez', 'lot4_numero', 'lot4_surface_carrez', 'lot5_numero', 'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude', 'Adresse', 

In [11]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import unicodedata

# 1. Définir la liste des colonnes souhaitées
desired_columns = [
    'valeur_fonciere',
    'Surface_habitable_logement',
    'nombre_pieces_principales',
    'Etiquette_DPE',
    'code_departement',
    'transaction_year',
    'Type_bâtiment',        # variable de contrôle (catégorielle)
    'Année_construction',   # variable de contrôle (numérique)
    'nature_mutation',      # variable de contrôle (catégorielle)
    'type_local',           # variable de contrôle (catégorielle)
    'surface_terrain',      # optionnelle
    'nombre_lots',          # optionnelle
    'surface_reelle_bati'   # optionnelle
]

# 2. Conserver seulement les colonnes disponibles dans la DataFrame originale "data"
available_columns = [col for col in desired_columns if col in data.columns]
print("Colonnes disponibles pour l'analyse :", available_columns)
df_working = data[available_columns].copy()

# 3. Créer les variables clés dans df_working
df_working['passoire'] = df_working['Etiquette_DPE'].apply(lambda x: 1 if x in ['F', 'G'] else 0)
df_working['post'] = df_working['transaction_year'].apply(lambda x: 1 if x >= 2021 else 0)
# Ne conserver que les transactions dont la valeur est positive
df_working = df_working[df_working['valeur_fonciere'] > 0].copy()
df_working['log_price'] = np.log(df_working['valeur_fonciere'])

# 4. Filtrer les données pour l'année 2022
df_2022_full = df_working[df_working['transaction_year'] == 2022].copy()

# 5. Pour les variables catégorielles qui serviront d'effets fixes, on force la conversion en chaîne
for var in ['code_departement', 'type_local', 'Type_bâtiment', 'nature_mutation']:
    if var in df_2022_full.columns:
        df_2022_full[var] = df_2022_full[var].astype(str)

# 6. Afin de traiter la variable "Année_construction" (qui contient un accent), 
# nous la renommons temporairement en "Annee_construction" si elle est présente.
if 'Année_construction' in df_2022_full.columns:
    df_2022_full.rename(columns={"Année_construction": "Annee_construction"}, inplace=True)

# 7. (Optionnel) Normaliser les noms de colonnes pour supprimer tous accents dans df_2022_full
df_2022_full.columns = [unicodedata.normalize('NFKD', col).encode('ascii', errors='ignore').decode('utf8') for col in df_2022_full.columns]
print("\nColonnes après normalisation :", df_2022_full.columns.tolist())

# 8. Construction dynamique de la formule du modèle :
# On démarre avec les variables de base
formula_terms = [
    "log_price ~ passoire",
    "Surface_habitable_logement",
    "nombre_pieces_principales"
]

# Ajouter la variable "Annee_construction" si présente (remplacera "Année_construction")
if 'Annee_construction' in df_2022_full.columns:
    formula_terms.append("Annee_construction")

# Ajouter les variables optionnelles si elles existent
if 'surface_terrain' in df_2022_full.columns:
    formula_terms.append("surface_terrain")
if 'surface_reelle_bati' in df_2022_full.columns:
    formula_terms.append("surface_reelle_bati")
if 'nombre_lots' in df_2022_full.columns:
    formula_terms.append("nombre_lots")

# Ajouter les effets fixes sur les variables catégorielles
for var in ['Type_batiment', 'nature_mutation', 'type_local', 'code_departement']:
    if var in df_2022_full.columns:
        formula_terms.append(f"C({var})")

# Construire la formule finale
formula_full = " + ".join(formula_terms)
print("\nFormule du modèle :", formula_full)

# 9. Ajuster le modèle OLS avec correction d'hétéroscédasticité (HC3)
model_full = smf.ols(formula=formula_full, data=df_2022_full).fit(cov_type='HC3')
print("\n=== Résultats de la régression statique pour 2022 avec variables de contrôle supplémentaires ===")
print(model_full.summary())


Colonnes disponibles pour l'analyse : ['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_bâtiment', 'Année_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati']

Colonnes après normalisation : ['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_batiment', 'Annee_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati', 'passoire', 'post', 'log_price']

Formule du modèle : log_price ~ passoire + Surface_habitable_logement + nombre_pieces_principales + Annee_construction + surface_terrain + surface_reelle_bati + nombre_lots + C(Type_batiment) + C(nature_mutation) + C(type_local) + C(code_departement)

=== Résultats de la régression statique pour 2022 avec variables de contrôle supplémentaires ===


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 96, but rank is 95
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.556
Model:                            OLS   Adj. R-squared:                  0.555
Method:                 Least Squares   F-statistic:                     376.5
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        08:23:20   Log-Likelihood:                -47635.
No. Observations:               43584   AIC:                         9.546e+04
Df Residuals:                   43488   BIC:                         9.630e+04
Df Model:                          95                                         
Covariance Type:                  HC3                                         
                                                                coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------

### Analyse Variables Principales

- **passoire**  
  - **Coefficient brut** : -0.2210  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{-0.2210} - 1) \approx -19.9\%\)  
  - **Conclusion** : Un bien classé comme passoire thermique (DPE F ou G) se vend environ 20 % moins cher qu’un bien performant, toutes choses égales par ailleurs. Cet effet est hautement significatif.

- **nombre_pieces_principales**  
  - **Coefficient brut** : 0.0441  
  - **p-value** : 0.050 (limite de significativité)  
  - **Interprétation en %** : \(100 \times (e^{0.0441} - 1) \approx 4.5\%\)  
  - **Conclusion** : Une augmentation d'une pièce principale est associée à une hausse d’environ 4.5 % du prix, toutes choses égales par ailleurs. Ce résultat est à la limite de la significativité (p = 0.050).

- **Annee_construction** (anciennement “Année_construction”)  
  - **Coefficient brut** : 0.0050  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{0.0050} - 1) \approx 0.50\%\)  
  - **Conclusion** : Chaque année supplémentaire (ou plus précisément, pour un bien plus récent) augmente le prix d’environ 0.5 %, toutes choses égales. Cet effet est statistiquement significatif.

- **surface_terrain**  
  - **Coefficient brut** : 0.0002  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{0.0002} - 1) \approx 0.02\%\)  
  - **Conclusion** : Bien que très significatif statistiquement, l’impact de la surface du terrain sur le log du prix est extrêmement faible en termes de variation en pourcentage.


- **nombre_lots**  
  - **Coefficient brut** : -0.4453  
  - **p-value** : < 0.001  
  - **Interprétation en %** : \(100 \times (e^{-0.4453} - 1) \approx -35.9\%\)  
  - **Conclusion** : Pour un bien comportant un plus grand nombre de lots, le prix est en moyenne réduit d’environ 36 %, toutes choses égales par ailleurs, ce qui peut refléter la nature des biens vendus en copropriété ou en plusieurs unités.

---

### Variables Catégorielles (Effets Fixes)

Les variables catégorielles sont intégrées via des effets fixes (avec la notation `C(variable)`). Chaque coefficient pour ces variables représente l’effet du changement de catégorie par rapport à la catégorie de référence (celle omise automatiquement par le modèle).

- **Type_batiment**  
  - Par exemple, le coefficient pour `C(Type_batiment)[T.maison]` est de -0.4836 (p < 0.001).  
  - **Interprétation en %** : \(100 \times (e^{-0.4836} - 1) \approx -38.35\%\)  
  - **Conclusion** : Être une maison (par rapport à la catégorie de référence, par exemple un appartement) est associé à une décote d’environ 38 %, ce qui peut être lié à des différences de localisation ou de caractéristiques structurelles.

- **nature_mutation**  
  - Les catégories présentent des effets variables. Par exemple, pour `C(nature_mutation)[T.Vente]`, le coefficient est 0.7073 (p < 0.001), ce qui correspond à une prime d’environ 102.84 % (calculée via \(100 \times (e^{0.7073} - 1)\)).  
  - Pour `C(nature_mutation)[T.Echange]`, le coefficient est de -1.1589 mais avec une p-value de 0.094, indiquant une tendance négative sans être significative au niveau de 5 %.  
  - **Conclusion** : La nature de la transaction influence les prix, avec certaines formes (comme la vente classique) associées à des primes, tandis que d'autres (comme l'échange) pourraient être associées à des décotes, bien que ce dernier effet ne soit pas très robuste ici.

- **type_local**  
  - Par exemple, `C(type_local)[T.Local industriel. commercial ou assimilé]` présente un coefficient de -0.4281 (p < 0.001), soit une variation d’environ -34.9 %.  
  - Pour `C(type_local)[T.Maison]`, le coefficient est de -0.9248 (p < 0.001), ce qui correspond à une décote d’environ -60.4 %.  
  - **Conclusion** : Le type local influence fortement la valorisation, avec certains types affichant des décotes importantes.

- **code_departement**  
  - Les effets fixes par département montrent des coefficients très variés. Par exemple, `C(code_departement)[T.13]` est positif (≈ 0.7787, p < 0.001) traduisant une prime régionale significative (environ +78.7 % d’augmentation relative), tandis que d’autres départements affichent des effets négatifs.  
  - **Conclusion** : Les disparités géographiques sont très marquées, et le département dans lequel se trouve le bien joue un rôle crucial dans la valorisation.

---

# Analyse Diff-in-Diff de l'Impact de la Loi Anti‑Passoire Thermique

L'objectif de cette analyse est d'estimer l'effet causal de la loi anti‑passoire thermique sur le prix des biens immobiliers. Pour ce faire, nous utiliserons une approche Difference‑in‑Differences (Diff‑in‑Diff) en panel couvrant plusieurs années (par exemple, 2019 à 2022).

### Hypothèses et stratégie d'identification

- **Groupe traité** : Les biens classés comme passoires thermiques (*passoire* = 1).
- **Groupe de contrôle** : Les biens dont la performance énergétique est supérieure (passoire = 0).
- **Variable "post"** : Nous définissons la période post‑intervention comme les transactions à partir de 2021, c'est-à-dire `post` = 1 pour l'après‑loi et 0 pour l'avant‑loi.
  
Le modèle Diff‑in‑Dif s'écrit de manière générale :
\[
\log(price)_{it} = \alpha + \beta_1 \, \text{passoire}_i + \gamma \, \text{post}_t + \delta \, (\text{passoire}_i \times \text{post}_t) + X_{it}\theta + \varepsilon_{it},
\]
où \(X_{it}\) regroupe les variables de contrôle (comme Surface_habitable_logement, nombre_pieces_principales, Annee_construction, etc.) et éventuellement des effets fixes (par département, année, etc.).

**Interprétation :**  
Le coefficient \(\delta\) de l'interaction \( \text{passoire}_i \times \text{post}_t\) capture l'effet additionnel (causal) de la loi sur le log des prix des biens classés comme passoires thermiques, par rapport au groupe de contrôle.


## Préparation des Données pour l'Analyse Diff-in-Diff

Nous allons utiliser l'ensemble de notre panel de données (par exemple, de 2019 à 2022) qui est stocké dans `df_working`.  
Nous vérifierons que la variable `post` est bien définie de manière à indiquer l'après‑loi (par exemple, post = 1 pour les transactions de 2021 et 2022, et 0 pour les années antérieures).  
De plus, nous conserverons les variables de contrôle utilisées précédemment et ajouterons des effets fixes temporels afin de contrôler pour les tendances globales du marché.


In [12]:
# Considérons que df_working a déjà été construit et contient les années 2019 à 2022.
# Vérifions d'abord la répartition des années
print("Répartition des transactions par année :")
print(df_working['transaction_year'].value_counts().sort_index())

# Pour cette analyse, nous pouvons définir post=1 pour les années >=2021
# (Si ce n'est pas déjà fait, la variable "post" doit déjà être dans df_working)
# Nous pouvons également créer des effets fixes temporels en incluant C(transaction_year)

# Créer un panel complet pour l'analyse Diff-in-Diff (de 2019 à 2022)
df_panel = df_working.copy()

# Afficher quelques lignes pour vérifier
print(df_panel.head())


Répartition des transactions par année :
transaction_year
2019     59549
2020     70931
2021     59549
2022    175485
Name: count, dtype: int64
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement  transaction_year Type_bâtiment  \
0             F               48              2019        maison   
1             G               48              2019        maison   
2             E               48              2019        maison   
3             E               48              2019   appartement   
4             E               48              2019   

## Spécification du Modèle Diff-in-Diff

Nous allons spécifier le modèle de la manière suivante :

\[
\begin{aligned}
\log(price)_{it} &= \alpha + \beta_1 \, \text{passoire}_i + \gamma \, \text{post}_t + \delta (\text{passoire}_i \times \text{post}_t)\\[0.5em]
&\quad + \theta_1\,\text{Surface\_habitable\_logement}_{it} + \theta_2\,\text{nombre\_pieces\_principales}_{it} + \theta_3\,\text{Annee\_construction}_{it} \\
&\quad + \text{(Effets fixes)} + \varepsilon_{it}
\end{aligned}
\]

Nous ajouterons également des effets fixes pour :
- La variable temporelle `transaction_year` (C(transaction_year)) afin de contrôler les tendances du marché,  
- Et pour la localisation via `code_departement` (C(code_departement)).

Le coefficient \(\delta\) (associé à l'interaction `passoire:post`) capture l'effet causal de la loi anti‑passoire thermique sur le prix immobilier.


In [15]:
# Vérifier la distribution de la variable 'post'
print("Distribution de 'post':")
print(df_panel['post'].value_counts())


Distribution de 'post':
post
1    235034
0    130480
Name: count, dtype: int64


In [24]:
# Supposons que df_panel_clean est créé en nettoyant df_panel (par exemple par dropna sur les variables requises)
required_vars = ['log_price', 'passoire', 'post', 
                 'Surface_habitable_logement', 'nombre_pieces_principales', 
                 'Annee_construction', 'transaction_year', 'code_departement']

df_panel_clean = df_panel.dropna(subset=required_vars).copy()

# Vérifier l'aperçu du DataFrame nettoyé
print("Aperçu de df_panel_clean :")
print(df_panel_clean.head())

# Afficher la liste complète des colonnes pour être sûr que tout est présent
print("\nColonnes dans df_panel_clean :")
print(df_panel_clean.columns.tolist())

# Vérifier la distribution de la variable transaction_year
print("\nRépartition de transaction_year :")
print(df_panel_clean['transaction_year'].value_counts())

# Vérifier la distribution de la variable code_departement
print("\nRépartition de code_departement :")
print(df_panel_clean['code_departement'].value_counts())


Aperçu de df_panel_clean :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
0         150000.0                       66.70                        3.0   
1         150000.0                       80.00                        4.0   
2         360500.0                      149.87                        5.0   
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   

  Etiquette_DPE code_departement transaction_year Type_batiment  \
0             F               48             2019        maison   
1             G               48             2019        maison   
2             E               48             2019        maison   
3             E               48             2019   appartement   
4             E               48             2019   appartement   

   Annee_construction nature_mutation   type_local  surface_terrain  \
0              1998.0           Vent

In [25]:
# Afficher la liste complète des colonnes pour être sûr que tout est présent
print("\nColonnes dans df_panel_clean :")
print(df_panel_clean.columns.tolist())

# Vérifier la distribution de la variable transaction_year
print("\nRépartition de transaction_year :")
print(df_panel_clean['transaction_year'].value_counts())

# Vérifier la distribution de la variable code_departement
print("\nRépartition de code_departement :")
print(df_panel_clean['code_departement'].value_counts())


Colonnes dans df_panel_clean :
['valeur_fonciere', 'Surface_habitable_logement', 'nombre_pieces_principales', 'Etiquette_DPE', 'code_departement', 'transaction_year', 'Type_batiment', 'Annee_construction', 'nature_mutation', 'type_local', 'surface_terrain', 'nombre_lots', 'surface_reelle_bati', 'passoire', 'post', 'log_price']

Répartition de transaction_year :
transaction_year
2022    133997
2020     70931
2019     59549
2021     59549
Name: count, dtype: int64

Répartition de code_departement :
code_departement
31    19628
44    13968
34    13795
92    13392
69    11033
      ...  
55      236
55      164
2A       86
48       69
2B       54
Name: count, Length: 108, dtype: int64


In [29]:
df_panel_clean['transaction_year'] = df_panel_clean['transaction_year'].astype(str)
df_panel_clean['code_departement'] = df_panel_clean['code_departement'].astype(str)


In [30]:
formula_test = "log_price ~ passoire + post + C(transaction_year) + C(code_departement)"
print("Formule test :", formula_test)

model_test = smf.ols(formula=formula_test, data=df_panel_clean, missing='drop').fit(cov_type='HC3')
print(model_test.summary())


Formule test : log_price ~ passoire + post + C(transaction_year) + C(code_departement)


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 87, but rank is 1
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.190
Method:                 Least Squares   F-statistic:                     1.784
Date:                Sun, 13 Apr 2025   Prob (F-statistic):              0.182
Time:                        08:36:49   Log-Likelihood:            -3.6027e+05
No. Observations:              324026   AIC:                         7.207e+05
Df Residuals:                  323939   BIC:                         7.216e+05
Df Model:                          86                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [32]:
# Étape 3 : Estimer le modèle Diff-in-Differences complet en développant l'interaction explicitement
formula_diff_test = (
    "log_price ~ passoire + post + passoire:post + Surface_habitable_logement + nombre_pieces_principales + "
    "Annee_construction + C(transaction_year) + C(code_departement)"
)
print("Formule Diff-in-Differences (test avec interaction développée) :", formula_diff_test)

# Estimer le modèle sur df_panel_clean en forçant l'ignorance des NA
model_diff_test = smf.ols(formula=formula_diff_test, data=df_panel_clean, missing='drop').fit(cov_type='HC3')
print("\n=== Résultats du modèle Diff-in-Differences (test) ===")
print(model_diff_test.summary())


Formule Diff-in-Differences (test avec interaction développée) : log_price ~ passoire + post + passoire:post + Surface_habitable_logement + nombre_pieces_principales + Annee_construction + C(transaction_year) + C(code_departement)

=== Résultats du modèle Diff-in-Differences (test) ===


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 91, but rank is 90
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     1818.
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        08:40:10   Log-Likelihood:            -3.2338e+05
No. Observations:              324026   AIC:                         6.469e+05
Df Residuals:                  323935   BIC:                         6.479e+05
Df Model:                          90                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

# Analyse des Résultats Diff‑in‑Differences

## Contexte et Objectif

Cette analyse Diff‑in‑Differences a été réalisée couvrant les années 2019 à 2022 dans le but d'identifier l'impact causal de la loi anti‑passoire thermique sur le prix des biens immobiliers.  
Nous comparons l'évolution des prix (exprimés en logarithme, `log_price`) entre deux périodes :
- **Période pré-intervention** : 2019–2020 (post = 0)
- **Période post-intervention** : 2021–2022 (post = 1)

Les biens sont identifiés comme **passoires** (passoire = 1) ou non‑passoires (passoire = 0). Le modèle Diff‑in‑Differences inclut une interaction entre le statut "passoire" et la période "post" pour capter l'impact spécifique de la loi sur les biens énergétiquement inefficaces.

## Spécification du Modèle

Le modèle estimé est le suivant :

\[
\begin{aligned}
\log(price)_{it} &= \alpha + \beta_1\,\text{passoire}_i + \gamma\,\text{post}_t + \delta\,(\text{passoire}_i \times \text{post}_t)\\[0.5em]
&\quad + \theta_1\,\text{Surface\_habitable\_logement}_{it} + \theta_2\,\text{nombre\_pieces\_principales}_{it} + \theta_3\,\text{Annee\_construction}_{it} \\[0.5em]
&\quad + \text{FE}_{\text{transaction\_year}} + \text{FE}_{\text{code\_departement}} + \varepsilon_{it}.
\end{aligned}
\]

Les effets fixes sur `transaction_year` et `code_departement` permettent de contrôler les tendances temporelles globales et les disparités géographiques.

## Résultats Clés

- **Effet de la variable *passoire***  
  - **Coefficient brut** : -0.0937 (p < 0.001)  
  - **Interprétation** : \(100 \times (e^{-0.0937} - 1) \approx -8.96\%\)  
  - **Conclusion** : Avant l'intervention, les biens identifiés comme passoires se vendent environ 9 % moins cher que les biens non‑passoires, toutes choses égales par ailleurs.

- **Effet de la variable *post***  
  - **Coefficient brut** : 0.0583 (p < 0.001)  
  - **Interprétation** : \(100 \times (e^{0.0583} - 1) \approx 6.00\%\)  
  - **Conclusion** : Globalement, après l'intervention, les prix augmentent d'environ 6 % pour tous les biens (tendance générale du marché).

- **Effet de l'interaction (*passoire × post*)**  
  - **Coefficient brut** : -0.0008 (p = 0.900)  
  - **Conclusion** : L'interaction, qui devrait capturer l'effet spécifique de la loi pour les biens passoires, n'est pas significative. Cela suggère qu'après l'intervention, il n'existe pas d'effet différentiel additionnel sur le log des prix pour les biens passoires par rapport aux biens non‑passoires, au-delà des effets principaux et de la tendance générale du marché.

- **Contrôles**  
  - **Surface_habitable_logement** : Un coefficient de 0.0089 (p < 0.001) indique qu'une augmentation d'une unité de surface est associée à une hausse modeste du prix (≈ 0.89 %).  
  - **nombre_pieces_principales** : Le coefficient est de 0.0290 (p ≈ 0.089), suggérant un effet positif à la limite de la significativité.  
  - **Annee_construction** : Le coefficient est significatif, bien que très faible (≈ 0.0050), indiquant qu'un bien plus récent se vend légèrement mieux.
  
- **Effets fixes**  
  Les effets fixes montrent des disparités importantes :
  - La variable `transaction_year` présente des différences significatives entre les années.
  - La variable `code_departement` capte de fortes disparités régionales (certains départements affichent des primes, d'autres une décote).

## Conclusion Globale

Le modèle Diff‑in‑Differences nous permet d'estimer l'effet causal de la loi anti‑passoire thermique sur le prix immobilier.  
- **Avant l'intervention**, les biens passoires se vendent environ 9 % moins cher.
- **Après l'intervention**, une augmentation générale d’environ 6 % est observée dans le marché, mais l'interaction qui devrait représenter l'impact spécifique de la loi sur les biens passoires n'est pas significative.  
Cela peut indiquer que, dans l'échantillon et la spécification actuels, l'effet différentiel de la loi sur les biens passoires n'est pas détecté ou qu'il est masqué par d'autres facteurs (par exemple, une forte multicolinéarité avec les effets fixes ou des spécifications de contrôle nécessitant des ajustements).


In [33]:
formula_diff_alt = ("log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + "
                    "Annee_construction + transaction_year + C(code_departement)")
model_diff_alt = smf.ols(formula=formula_diff_alt, data=df_panel_clean, missing='drop').fit(cov_type='HC3')
print(model_diff_alt.summary())


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 91, but rank is 90
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     1818.
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        08:52:46   Log-Likelihood:            -3.2338e+05
No. Observations:              324026   AIC:                         6.469e+05
Df Residuals:                  323935   BIC:                         6.479e+05
Df Model:                          90                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [34]:
formula_diff_alt = ("log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + "
                    "Annee_construction + transaction_year + C(code_departement)")
model_diff_alt = smf.ols(formula=formula_diff_alt, data=df_panel_clean, missing='drop').fit(cov_type='HC3')
print(model_diff_alt.summary())


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 91, but rank is 90
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     1818.
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        08:54:57   Log-Likelihood:            -3.2338e+05
No. Observations:              324026   AIC:                         6.469e+05
Df Residuals:                  323935   BIC:                         6.479e+05
Df Model:                          90                                         
Covariance Type:                  HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

## Filtrage par Type de Bien : Focus sur les Appartements

La colonne **Type_batiment** de notre DataFrame contient trois valeurs : "maison", "appartement" et "immeuble".  
Pour réduire l'hétérogénéité de l'échantillon et se concentrer sur une sous-population homogène, nous allons filtrer les données pour ne conserver que les appartements.  

Le code ci-dessus réalise ce filtrage en sélectionnant les lignes où la valeur de **Type_batiment** (convertie en minuscules pour éviter tout problème de casse) est égale à "appartement".  
Après cette opération, nous utiliserons ce sous-échantillon (`df_appart`) pour poursuivre l'analyse Diff‑in‑Differences.


In [36]:
# Vérifiez les valeurs uniques dans la colonne Type_batiment
print("Valeurs uniques de Type_batiment dans df_working :")
print(df_working['Type_bâtiment'].unique())


Valeurs uniques de Type_batiment dans df_working :
['maison' 'appartement' 'immeuble']


In [38]:
# Filtrer le DataFrame pour ne conserver que les appartements
df_appart = df_working[df_working['Type_bâtiment'].str.lower() == 'appartement'].copy()

# Afficher un aperçu du sous-échantillon
print("Aperçu du DataFrame pour les appartements :")
print(df_appart.head())

# Vérifier la distribution de la variable 'Type_batiment' dans ce sous-échantillon
print("\nValeurs uniques de Type_batiment dans df_appart :")
print(df_appart['Type_bâtiment'].unique())


Aperçu du DataFrame pour les appartements :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   
5          87000.0                       70.00                        4.0   
6         160000.0                       82.35                        3.0   
7         266000.0                      101.03                        4.0   

  Etiquette_DPE code_departement  transaction_year Type_bâtiment  \
3             E               48              2019   appartement   
4             E               48              2019   appartement   
5             F               48              2019   appartement   
6             E               2A              2019   appartement   
7             D               2A              2019   appartement   

   Année_construction nature_mutation   type_local  surface_terrain  \
3            

In [40]:
# Filtrer le DataFrame df_working pour ne conserver que les appartements (insensible à la casse)
df_appart = df_working[df_working['Type_bâtiment'].str.lower() == 'appartement'].copy()

# Vérifier l'aperçu et la distribution
print("Aperçu du DataFrame pour les appartements :")
print(df_appart.head())
print("\nValeurs uniques de Type_batiment dans df_appart :")
print(df_appart['Type_bâtiment'].unique())

# Il est important de conserver les autres variables requises :
required_vars = ['log_price', 'passoire', 'post', 
                 'Surface_habitable_logement', 'nombre_pieces_principales', 
                 'Année_construction', 'transaction_year', 'code_departement']

# Nettoyer les lignes manquantes sur ces variables
df_appart_clean = df_appart.dropna(subset=required_vars).copy()

# Renommer "Année_construction" en "Annee_construction" pour éviter les problèmes d'accents
if 'Année_construction' in df_appart_clean.columns:
    df_appart_clean.rename(columns={"Année_construction": "Annee_construction"}, inplace=True)

# Normaliser les noms de colonnes (optionnel)
import unicodedata
df_appart_clean.columns = [unicodedata.normalize('NFKD', col).encode('ascii', errors='ignore').decode('utf8')
                          for col in df_appart_clean.columns]
print("\nColonnes dans df_appart_clean :", df_appart_clean.columns.tolist())

# Forcer la conversion de transaction_year et code_departement en chaînes
df_appart_clean['transaction_year'] = df_appart_clean['transaction_year'].astype(str)
df_appart_clean['code_departement'] = df_appart_clean['code_departement'].astype(str)

# Vérifier la distribution pour s'assurer de la variabilité
print("\nRépartition de transaction_year dans df_appart_clean :")
print(df_appart_clean['transaction_year'].value_counts())
print("\nRépartition de code_departement dans df_appart_clean :")
print(df_appart_clean['code_departement'].value_counts())


Aperçu du DataFrame pour les appartements :
   valeur_fonciere  Surface_habitable_logement  nombre_pieces_principales  \
3          85000.0                       29.30                        2.0   
4          85000.0                       29.30                        2.0   
5          87000.0                       70.00                        4.0   
6         160000.0                       82.35                        3.0   
7         266000.0                      101.03                        4.0   

  Etiquette_DPE code_departement  transaction_year Type_bâtiment  \
3             E               48              2019   appartement   
4             E               48              2019   appartement   
5             F               48              2019   appartement   
6             E               2A              2019   appartement   
7             D               2A              2019   appartement   

   Année_construction nature_mutation   type_local  surface_terrain  \
3            

In [41]:
# Définir la formule Diff-in-Differences pour les appartements
formula_diff_app = (
    "log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + "
    "Annee_construction + C(transaction_year) + C(code_departement)"
)
print("\nFormule Diff-in-Differences pour les appartements :", formula_diff_app)

# Estimer le modèle OLS avec correction d'hétéroscédasticité (HC3)
model_diff_app = smf.ols(formula=formula_diff_app, data=df_appart_clean, missing='drop').fit(cov_type='HC3')
print("\n=== Résultats du Modèle Diff-in-Differences pour les appartements ===")
print(model_diff_app.summary())



Formule Diff-in-Differences pour les appartements : log_price ~ passoire * post + Surface_habitable_logement + nombre_pieces_principales + Annee_construction + C(transaction_year) + C(code_departement)

=== Résultats du Modèle Diff-in-Differences pour les appartements ===


/opt/conda/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 91, but rank is 90
  warnings.warn('covariance of constraints does not have full '


                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     1394.
Date:                Sun, 13 Apr 2025   Prob (F-statistic):               0.00
Time:                        09:01:52   Log-Likelihood:            -2.4278e+05
No. Observations:              230631   AIC:                         4.857e+05
Df Residuals:                  230540   BIC:                         4.867e+05
Df Model:                          90                                         
Covariance Type:                  HC3                                         
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

# Analyse Diff‑in‑Differences pour les Appartements

## Contexte et Objectif

Cette analyse se concentre exclusivement sur le segment des appartements. Nous cherchons à estimer l’impact causal de la loi anti‑passoire thermique sur le prix des appartements en utilisant une approche Diff‑in‑Differences (Diff‑in‑Dif).  
Pour cela, nous comparons l’évolution des prix (exprimés en logarithme, `log_price`) entre les périodes pré‑et post‑intervention pour deux groupes :

- **Groupe traité (passoire = 1)** : Appartements classés comme passoires thermiques (DPE = F ou G).  
- **Groupe de contrôle (passoire = 0)** : Appartements mieux classés énergétiquement.

La variable **post** est codée comme suit :
- **post = 0** pour les années 2019–2020 (période pré‑intervention)
- **post = 1** pour les années 2021–2022 (période post‑intervention)

## Spécification du Modèle

Le modèle Diff‑in‑Differences est spécifié comme suit :

\[
\begin{aligned}
\log(price)_{it} =\; &\alpha + \beta_1\,\text{passoire}_i + \gamma\,\text{post}_t + \delta\,(\text{passoire}_i \times \text{post}_t)\\[1ex]
&+ \theta_1\,\text{Surface\_habitable\_logement}_{it} + \theta_2\,\text{nombre\_pieces\_principales}_{it} + \theta_3\,\text{Annee\_construction}_{it} \\
&+ \text{FE}_{\text{transaction\_year}} + \text{FE}_{\text{code\_departement}} + \varepsilon_{it}.
\end{aligned}
\]

Les effets fixes sur *transaction_year* et *code_departement* contrôlent les tendances générales du marché et les disparités régionales.

## Résultats Clés et Interprétation

Les coefficients estimés (avec erreurs robustes HC3) pour les variables principales sont :

- **passoire** :  
  - Coefficient : -0.0640  
  - \(100 \times \left(e^{-0.0640} - 1\right) \approx -6.2\%\)  
  **Interprétation** : Avant l'intervention, les appartements classés comme passoires se vendent en moyenne environ 6 % moins cher que les appartements non‑passoires.

- **post** :  
  - Coefficient : 0.0607  
  - \(100 \times \left(e^{0.0607} - 1\right) \approx +6.3\%\)  
  **Interprétation** : Pour les appartements non‑passoires (groupe de contrôle), la période post‑intervention est associée à une augmentation globale des prix d’environ 6 %, reflétant une tendance générale du marché.

- **Interaction (passoire × post)** :  
  - Coefficient : 0.0597  
  - \(100 \times \left(e^{0.0597} - 1\right) \approx +6.17\%\)  
  **Interprétation** : L'interaction indique que, dans la période post‑intervention, l'écart de prix entre les appartements passoires et non‑passoires est réduit de 0.0640 (niveau initial négatif) de 0.0597 points, soit un effet net de \(-0.0640 + 0.0597 \approx -0.0043\) en log, ce qui est pratiquement nul.  
  **Conclusion** : L'effet différentiel de la loi pour les appartements passoires semble atténuer la décote initiale de 6 %, de sorte qu'après l'intervention, la différence de prix entre passoires et non‑passoires devient quasiment inexistante.

- **Contrôles** :  
  - **Surface_habitable_logement** : Coefficient de 0.0147, indiquant qu'une augmentation d'une unité de surface correspond à une hausse d'environ 1.47 % du prix.
  - **nombre_pieces_principales** : Coefficient de -0.0462, suggérant qu'une augmentation d'une pièce est associée à une baisse d’environ 4.6 % (ce résultat peut refléter des effets de renormalisation lorsque l'on se concentre sur un sous-échantillon homogène).
  - **Annee_construction** : Un effet positif (environ 0.0000162 par unité) indique qu’un appartement plus récent se vend légèrement mieux.

- **Effets fixes** :  
  Les effets fixes pour *transaction_year* et *code_departement* révèlent d’importantes disparités temporelles et régionales. Par exemple, on observe des différences significatives dans les coefficients associés aux différentes années et départements, ce qui confirme que la localisation et la période jouent un rôle crucial dans la valorisation.

## Conclusion

En se concentrant uniquement sur les appartements :

- **Avant l'intervention**, les appartements passoires se vendaient environ 6 % moins cher.
- **Après l'intervention**, l'effet différentiel pour les appartements passoires (capturé par l'interaction passoire × post) est positif, ce qui compense presque totalement la décote initiale.  
  Cela suggère que, dans le segment des appartements, la loi anti‑passoire semble avoir permis de réduire (voire d'annuler) la décote de prix associée aux biens énergétiquement inefficaces.

Ces résultats indiquent que l’impact spécifique de l’intervention (l'effet causal additionnel sur les passoires) n'est pas détectable dans ce modèle parce que, après la loi, l'écart de prix entre appartements passoires et non‑passoires devient pratiquement nul.  
Il reste possible d'explorer d'autres spécifications (par exemple, en ajustant les effets fixes ou en segmentant par région) pour vérifier la robustesse de cette conclusion.